# Strong Json
A more faithful json encoder,decoder.

In [1]:
# sometimes binder can be slow in getting the lastest lib
# !pip install --upgrade strong_json

In [2]:
from strong_json import strong_json, ToJsonable, StrongJson, JSONPrimitive
from datetime import date

# Simple Usage

## Encode JSON

In [3]:
obj = [1, 'hello', 2.3, 1.0]
strong_json.to_json(obj)

'[1, "hello", 2.3, 1.0]'

In [4]:
obj = {
    'list': [1,2,3],
    'str': 'hello',
    'tuple': (3,5),
    'nested': {'c.a':1, 'c.b':2, 'c.c':3},
    'date': date(2019, 8, 23)
}
obj_json = strong_json.to_json(obj, indent=2) # all the keyword argument are passed to json.dumps
print(obj_json)

{
  "__type__": "dict",
  "__data__": [
    {
      "key": "list",
      "value": [
        1,
        2,
        3
      ]
    },
    {
      "key": "str",
      "value": "hello"
    },
    {
      "key": "tuple",
      "value": {
        "__type__": "tuple",
        "__data__": [
          3,
          5
        ]
      }
    },
    {
      "key": "nested",
      "value": {
        "__type__": "dict",
        "__data__": [
          {
            "key": "c.a",
            "value": 1
          },
          {
            "key": "c.b",
            "value": 2
          },
          {
            "key": "c.c",
            "value": 3
          }
        ]
      }
    },
    {
      "key": "date",
      "value": {
        "__type__": "date",
        "year": 2019,
        "month": 8,
        "day": 23
      }
    }
  ]
}


In [5]:
# numpy is also supported
import numpy as np
import pandas as pd
a = np.array([1,2,3])
b = pd.DataFrame({'a':[1,2], 'b':[3,4]})
print(strong_json.to_json(a))
print(strong_json.to_json(b))

{"__type__": "numpy.ndarray", "__data__": [1, 2, 3]}
{"__type__": "pandas.DataFrame", "__data__": {"__type__": "dict", "__data__": [{"key": "a", "value": {"__type__": "dict", "__data__": [{"key": 0, "value": 1}, {"key": 1, "value": 2}]}}, {"key": "b", "value": {"__type__": "dict", "__data__": [{"key": 0, "value": 3}, {"key": 1, "value": 4}]}}]}}


### Custom Class Encoder

Custom class Encoder is supported out of the box

In [6]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    def __str__(self):
        return f"User({self.first_name}, {self.last_name})"
    def __repr__(self):
        return str(self)
    def __hash__(self):
        return hash((self.first_name, self.last_name))
       
obj = User('Hello', 'World')
got = strong_json.to_json(obj)
print(got)

{"__type__": "User", "first_name": "Hello", "last_name": "World"}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/strong_json/__init__.py:119: ClassMapLookUpFailWarning: User not found in class map. You will not be able to convert this back.
  ClassMapLookUpFailWarning)


In [7]:
obj = {User('Mario', 'Mario'): 'Main Character', User('Luigi', 'Mario'): 'Side Kick'}
got = strong_json.to_json(obj, indent=1)
print(got)

{
 "__type__": "dict",
 "__data__": [
  {
   "key": {
    "__type__": "User",
    "first_name": "Mario",
    "last_name": "Mario"
   },
   "value": "Main Character"
  },
  {
   "key": {
    "__type__": "User",
    "first_name": "Luigi",
    "last_name": "Mario"
   },
   "value": "Side Kick"
  }
 ]
}


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/strong_json/__init__.py:119: ClassMapLookUpFailWarning: User not found in class map. You will not be able to convert this back.
  ClassMapLookUpFailWarning)


### Custom Encoder

If the default custom class encoder doesn't work for you. You could just implement ```ToJsonable``` interface and override 
```
def to_json_dict(self, encoder: StrongJson) -> Dict[str, JSONPrimitive]:
```

In [8]:
class FullUser(ToJsonable):
    def __init__(self, first, last):
        self.first = first
        self.last = last
    def to_json_dict(self, encoder: StrongJson):
        return {
            '__type__': 'FullUser',
            'first': encoder.to_json_dict(self.first),
            'last': encoder.to_json_dict(self.last),
            'full_name': encoder.to_json_dict(f"{self.first} {self.last}")
        }
    
strong_json.to_json(FullUser('hello', 'world'))

'{"__type__": "FullUser", "first": "hello", "last": "world", "full_name": "hello world"}'

## Decode JSON

In [9]:
got = strong_json.from_json(obj_json)
from pprint import pprint
pprint(got) # pprint doesn't print dict in the correct order though
print(list(got.keys())) # the key is sorted correctly

{'date': datetime.date(2019, 8, 23),
 'list': [1, 2, 3],
 'nested': {'c.a': 1, 'c.b': 2, 'c.c': 3},
 'str': 'hello',
 'tuple': (3, 5)}
['list', 'str', 'tuple', 'nested', 'date']


In [10]:
# don't worry normal json dict is decoder is supported
strong_json.from_json('{"a": "b", "c":"d"}')

{'a': 'b', 'c': 'd'}

In [11]:
# numpy is also supported
import numpy as np
import pandas as pd
a = np.array([1,2,3])
a_json = strong_json.to_json(a)
print(a_json)
got = strong_json.from_json(a_json)
print(got)


{"__type__": "numpy.ndarray", "__data__": [1, 2, 3]}
[1 2 3]


In [12]:
b = pd.DataFrame({'a':[1,2], 'b':[3,4]})
b_json = strong_json.to_json(b)
print(b_json)

got = strong_json.from_json(b_json)
got

{"__type__": "pandas.DataFrame", "__data__": {"__type__": "dict", "__data__": [{"key": "a", "value": {"__type__": "dict", "__data__": [{"key": 0, "value": 1}, {"key": 1, "value": 2}]}}, {"key": "b", "value": {"__type__": "dict", "__data__": [{"key": 0, "value": 3}, {"key": 1, "value": 4}]}}]}}


,a,b
0,1,3
1,2,4


### Custom Class Decoder

In [13]:
from strong_json import ClassMapBuilder
class_map = ClassMapBuilder.build_class_map(
    [User]
)
class_map # just a simple dictionary
custom_json = StrongJson(class_map=class_map)

In [14]:
obj = {User('Mario', 'Mario'): 'Main Character', User('Luigi', 'Mario'): 'Side Kick'}
json_str = custom_json.to_json(obj)
print(json_str)
# decode it back
got_obj = custom_json.from_json(json_str)
pprint(got_obj)

{"__type__": "dict", "__data__": [{"key": {"__type__": "User", "first_name": "Mario", "last_name": "Mario"}, "value": "Main Character"}, {"key": {"__type__": "User", "first_name": "Luigi", "last_name": "Mario"}, "value": "Side Kick"}]}
{User(Luigi, Mario): 'Side Kick', User(Mario, Mario): 'Main Character'}


#### If you really cannot to use the mixin
You can subclass StrongJson and add the functionality

In [15]:
class MixinHater:
    def __init__(self, msg: str):
        self.msg = msg
        
    def __str__(self):
        return f'MixinHater({self.msg})'

        
class CustomJson(StrongJson):
    def to_json_dict(self, obj):
        if isinstance(obj, MixinHater):
            return {
                self.type_key: 'MixinHater',
                'msg': self.default_to_json_dict(obj.msg)
            }
        else:
            return self.default_to_json_dict(obj)
        
    def from_json_dict(self, d):
        if self.type_key in d and d[self.type_key] == 'MixinHater':
            return MixinHater(d['msg'])

custom_json=CustomJson(class_map={})

In [16]:
json_str = custom_json.to_json(MixinHater('why not'))

In [17]:
obj = custom_json.from_json(json_str)
print(obj)

MixinHater(why not)
